In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [17]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 0: cat ; 1: dog
n_cats=50
n_dogs=50

X=np.zeros((n_cats+n_dogs,3*50*50))
t=np.zeros(n_cats+n_dogs)

for i in range(n_cats):
    t[i]=0
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train/cat.'+str(i)+'.jpg')
    img=cv2.resize(img,dsize=(50,50))
    img=img.reshape(3*50*50)/255
    X[i]=img
    
    if (i+1)%1000==0:
        print('Now processing ',i,'-th image')
        
for i in range(n_cats):
    t[i+n_cats]=1
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train/dog.'+str(i+n_cats)+'.jpg')
    img=cv2.resize(img,dsize=(50,50))
    img=img.reshape(3*50*50)/255
    X[i+n_cats]=img
    
    if (i+1)%1000==0:
        print('Now processing ',i,'-th image')        

Now processing  999 -th image
Now processing  1999 -th image
Now processing  2999 -th image
Now processing  3999 -th image
Now processing  4999 -th image
Now processing  999 -th image
Now processing  1999 -th image
Now processing  2999 -th image
Now processing  3999 -th image
Now processing  4999 -th image


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, t_train, t_test =train_test_split(X, t, test_size=0.2,shuffle=True) 

In [20]:
# Logistic Regression
LR=LogisticRegression(C=1,solver='lbfgs',max_iter=1000000) 

LR.fit(X_train,t_train)

# C is a parameter for regularization. Regularization can be stronger as C ->+0
train_pred=LR.predict(X_train)
train_score=LR.score(X_train,t_train)
print(train_score)

test_pred=LR.predict(X_test)
test_score=LR.score(X_test,t_test)
print(test_score)

1.0
0.534


In [21]:
"""
# Neural Network
MLPC=MLPClassifier(hidden_layer_sizes=(1000,),activation='relu',max_iter=1000000,alpha=0)

MLPC.fit(X_train,t_train)

print('Finished training')

train_pred=MLPC.predict(X_train)
train_score=MLPC.score(X_train,t_train)
print(train_score)    

test_pred=MLPC.predict(X_test)
test_score=MLPC.score(X_test,t_test)
print(test_score)
"""

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.9245
0.592


In [23]:
import pandas as pd
n_test=12500
ID=range(1,12500+1)
Output=pd.DataFrame(index=ID)
Output.index.name='id'

X_submit=np.zeros((n_test,3*50*50))

for i in ID:
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test/'+str(i)+'.jpg')
    img=cv2.resize(img,dsize=(50,50))
    img=img.reshape(3*50*50)/255
    X_submit[i-1]=img
    
pred=LR.predict(X_submit)
print(pred)

[0. 1. 1. ... 1. 1. 0.]


In [24]:
pred=LR.predict_proba(X_submit)
print(pred)

[[0.51077244 0.48922756]
 [0.05990623 0.94009377]
 [0.26758881 0.73241119]
 ...
 [0.00264982 0.99735018]
 [0.03190331 0.96809669]
 [0.98983456 0.01016544]]


In [25]:
Output['label']=pred[:,1]
Output[0:10]

,label
id,
1,0.489228
2,0.940094
3,0.732411
4,0.917192
5,0.131339
6,0.865056
7,0.039414
8,0.000391
9,0.511372


In [26]:
Output.to_csv('MyOutput.csv')